In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
benchmarks = ['gcc', 'lbm', 'mcf', 'namd', 'xalancbmk']
complete_result = {}

In [17]:
def extract_counts(file_path):
    counters = {
        'mem_inst_retired.any': [], 
        'branch-instructions': [],
        'branch-misses': [], 
        'L1-dcache-loads': [], 
        'L1-dcache-load-misses': [], 
        'L2-loads': [],
        'L2-load-misses': [], 
        'L2-stores': [],
        'L2-store-misses': [], 
        'LLC-loads': [], 
        'LLC-load-misses': [], 
        'LLC-stores': [], 
        'LLC-store-misses': [],
        'mem-loads': [],
        'mem-stores': [],
        'cycles': [], 
        'instructions': [],
    }
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            value, key = line.split()[1:3]
            counters[key].append(float(value.replace(',', '')))
            
    # Sanity Check
    lengths = [len(counters[key]) for key in counters]
    for i in range(1, len(lengths)):
        assert lengths[i] == lengths[i-1]
    print(f'Number of Count samples: {lengths[0]}')
    return counters

def extract_power(file_path):
    counters = {
        'joule': [], 
        'time': []
    }
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            time, joule = line.split()[0:2]
            counters['joule'].append(float(joule))
            counters['time'].append(float(time))
    
    # Sanity Check
    lengths = [len(counters[key]) for key in counters]
    for i in range(1, len(lengths)):
        assert lengths[i] == lengths[i-1]
    print(f'Number of Power samples: {lengths[0]}')
    return counters

def extract_results(counters):
    results = {
        'IPC': [],
        'branch-pred-acc': [],
        'L1-cache-acc': [],
        'L2-cache-acc': [],
        'L3-cache-acc': [],
        'L1-hit-rate': [],
        'L2-hit-rate': [],
        'L3-hit-rate': [],
        'memory-reqs': [],
        'power (watt)': [],
    }
    normal_len = len(counters['instructions'])
    for i in range(normal_len):
        results['IPC'].append(counters['instructions'][i] / counters['cycles'][i])
        results['branch-pred-acc'].append(1 - counters['branch-misses'][i] / counters['branch-instructions'][i])
        results['L1-cache-acc'].append(counters['L1-dcache-loads'][i])
        results['L2-cache-acc'].append(counters['L2-loads'][i] + counters['L2-stores'][i])
        results['L3-cache-acc'].append(counters['LLC-loads'][i] + counters['LLC-stores'][i])
        results['L1-hit-rate'].append(1 - counters['L1-dcache-load-misses'][i] / results['L1-cache-acc'][i])
        results['L2-hit-rate'].append(1 - ( counters['L2-load-misses'][i]+counters['L2-store-misses'][i] ) / results['L2-cache-acc'][i])
        results['L3-hit-rate'].append(1 - ( counters['LLC-load-misses'][i]+counters['LLC-store-misses'][i] ) / results['L3-cache-acc'][i])
        results['memory-reqs'].append(counters['mem-loads'][i]+counters['mem-stores'][i])
    
    power_len = len(counters['joule'])
    for i in range(power_len-2):
        results['power (watt)'].append(counters['joule'][i] / (counters['time'][i+1] - counters['time'][i]))
        
    return results

def smooth(arr, window):
    smoothed = []
    for i in range(len(arr)):
        left = int(max(0, i - window // 2))
        right = int(min(len(arr), i + window // 2 + 1))
        window_values = arr[left:right]
        smoothed_value = sum(window_values) / len(window_values)
        smoothed.append(smoothed_value)
    return smoothed


In [18]:
for benchmark in benchmarks:
    print("--------------------------")
    print(f"Processing {benchmark}......")
    counters = extract_counts(f'./math/{benchmark}-math.count')
    power_counters = extract_power(f'./power/{benchmark}-phy.count')
    counters.update(power_counters)
    complete_result[benchmark] = extract_results(counters)

--------------------------
Processing gcc......
Number of Count samples: 231
Number of Power samples: 234
--------------------------
Processing lbm......
Number of Count samples: 440
Number of Power samples: 435
--------------------------
Processing mcf......
Number of Count samples: 196
Number of Power samples: 193
--------------------------
Processing namd......
Number of Count samples: 394
Number of Power samples: 388
--------------------------
Processing xalancbmk......
Number of Count samples: 1095
Number of Power samples: 1108


In [9]:
def plot_and_save_graphs(metric, smooth_ratio):
    plt.figure(figsize=(15, 10))
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    for benchmark, result in complete_result.items():
        n = len(result[metric])
        window = n*smooth_ratio
        plt.plot([50*i for i in range(n)], smooth(result[metric], window), label=benchmark)
    plt.legend(fontsize=20)
    plt.title(f"Plot of {metric} for Different Benchmarks", fontsize=20)
    plt.xlabel("Time in (ms)", fontsize=20)
    plt.ylabel(f"{metric}", fontsize=20)
    plt.tight_layout()
    plt.savefig(f"plots/{metric}.png")
    plt.close()

In [14]:
for key in complete_result['gcc'].keys():
    plot_and_save_graphs(metric=key, smooth_ratio=0.08)